In [12]:
gap_score = -1
match_score = +1
mismatch_score = -1
call_counter = 0

In [53]:
def align(seq1, seq2):
    global call_counter
    call_counter += 1
    if not seq1:
        return "-" * len(seq2), seq2, len(seq2) * gap_score
    
    if not seq2:
        return seq1, "-" * len(seq1), len(seq1) * gap_score
    
    aligned1_left, aligned2_left, score_left = align(seq1, seq2[:-1])
    aligned1_left_result = aligned1_left + "-"
    aligned2_left_result = aligned2_left + seq2[-1]
    score_left = score_left + gap_score
    
    result_left = [score_left, aligned1_left_result, aligned2_left_result]
    
    aligned1_right, aligned2_right, score_right = align(seq1[:-1], seq2)
    aligned1_right_result = aligned1_right + seq1[-1]
    aligned2_right_result = aligned2_right + "-"
    score_right = gap_score + score_right
    
    result_right = [score_right, aligned1_right_result, aligned2_right_result]
    
    aligned1_middle, aligned2_middle, score_middle = align(seq1[:-1], seq2[:-1])
    
    aligned1_middle_result = aligned1_middle + seq1[-1]
    aligned2_middle_result = aligned2_middle + seq2[-1]
    
    if seq1[-1] == seq2[-1]:
        score_adjustment = match_score
    else:
        score_adjustment = mismatch_score
        
    score_middle = score_middle + score_adjustment
    result_middle = [score_middle, aligned1_middle_result, aligned2_middle_result]
    results = [result_left, result_right, result_middle]
    results.sort()
    result = results[-1]
    
    return result[1], result[2], result[0]

In [54]:
def align_fast(seq1, seq2, cache = None):
    global call_counter
    call_counter += 1
    if cache == None:
        cache = {}
        
    if not seq1:
        return "-" * len(seq2), seq2, len(seq2) * gap_score
    
    if not seq2:
        return seq1, "-" * len(seq1), len(seq1) * gap_score

    left_seqs = seq1, seq2[:-1]
    
    if left_seqs in cache:
        aligned1_left, aligned2_left, score_left = cache[left_seqs]
    else:
        aligned1_left, aligned2_left, score_left = align_fast(*left_seqs, cache)
        cache[left_seqs] = aligned1_left, aligned2_left, score_left
    
    aligned1_left_result = aligned1_left + "-"
    aligned2_left_result = aligned2_left + seq2[-1]
    score_left = score_left + gap_score
    result_left = (score_left, aligned1_left_result, aligned2_left_result)
    
    right_seqs = seq1[:-1], seq2
    
    if right_seqs in cache:
        aligned1_right, aligned2_right, score_right = cache[right_seqs]
    else:
        aligned1_right, aligned2_right, score_right = align_fast(*right_seqs, cache)
        cache[right_seqs] = aligned1_right, aligned2_right, score_right

    aligned1_right_result = aligned1_right + seq1[-1]
    aligned2_right_result = aligned2_right + "-"
    score_right = gap_score + score_right
    result_right = (score_right, aligned1_right_result, aligned2_right_result)
    
    mid_seqs = seq1[:-1], seq2[:-1]
    
    if mid_seqs in cache:
         aligned1_middle, aligned2_middle, score_middle = cache[mid_seqs]
    else:
        aligned1_middle, aligned2_middle, score_middle = align_fast(*mid_seqs, cache)
        cache[mid_seqs] = aligned1_middle, aligned2_middle, score_middle
    
    aligned1_middle_result = aligned1_middle + seq1[-1]
    aligned2_middle_result = aligned2_middle + seq2[-1]
    
    if seq1[-1] == seq2[-1]:
        score_adjustment = match_score
    else:
        score_adjustment = mismatch_score

    score_middle = score_middle + score_adjustment
    result_middle = (score_middle, aligned1_middle_result, aligned2_middle_result)
        
    cache[mid_seqs] = result_middle
        
    results = [result_left, result_right, result_middle]
    results.sort()
    result = results[-1]
    
    return result[1], result[2], result[0]

In [61]:
call_counter = 0
align("GCATGCTAA", "GATTACAAA")

('GCATG-CTAA', 'G-ATTACAAA', 2)

In [62]:
call_counter

2193844

In [48]:
gap_score = -1
match_score = +1
mismatch_score = -1

In [57]:
call_counter = 0
align_fast("GCATGCTGCATGCT", "GATTACAGATTACA")

('GCATG-CTGCATG-CT', 'G-ATTACAG-ATTACA', 0)

In [58]:
call_counter

225